### 使用zhipuai库来调用智谱大模型的API


#### 1. 创建客户端

In [12]:
from zhipuai import ZhipuAI
from rich import print
from dotenv import load_dotenv
from IPython.display import display, Markdown, Code
import os
load_dotenv()
api_key = os.getenv("ZHIPU_API_KEY")

client = ZhipuAI(api_key=api_key)

#### 2. 进行单轮对话

In [13]:
response_1= client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "什么是智能体"}
    ]
)
print(response_1.choices[0].message)

CompletionMessage(
    content='智能体（Agent）在人工智能领域是一个核心概念，指的是能够感知环境并做出行动以实现目标的实体。智能体可以
是软件程序、机器人，也可以是虚拟的实体。以下是对智能体的一些基本特征和类型：\n\n### 特征：\n\n1. 
**感知能力**：智能体能够从环境中获取信息。\n2. **决策能力**：基于感知到的信息，智能体能够做出决策。\n3. 
**行动能力**：智能体能够执行决策并作用于环境。\n4. 
**学习与适应能力**：智能体能够通过经验学习和适应环境的变化。\n\n### 类型：\n\n1. 
**软件智能体**：如计算机程序，能够执行特定任务。\n2. **硬件智能体**：如机器人，具有物理形态和执行任务的能力。\n3. 
**虚拟智能体**：存在于虚拟环境中的实体，如电子游戏中的角色。\n\n### 应用：\n\n- **搜索与导航**：如GPS导航系统。\n- 
**自动化系统**：如自动驾驶汽车。\n- 
**决策支持系统**：如金融市场的交易系统。\n\n智能体是人工智能研究和应用中的一个重要领域，它们在实现自动化、智能化等
方面发挥着关键作用。',
    role='assistant',
    reasoning_content=None,
    tool_calls=None
)

In [ ]:
response_2= client.chat.completions.create(
    model="glm-4",
    messages=[
        {"role": "user", "content": "你提到它是谁"}
    ]
)
print(response_2.choices[0].message.content)

很明显，模型并不记得之前的信息，对话与对话之间是相互独立的。如果我们想让模型记得之前的信息，就需要将上一个creaye方法产出的结果输入到下一个create方法中。所以在大模型开发的流程中，对话功能是嵌套的逻辑，而不是线性的逻辑

In [ ]:
mem_1 = response_1.choices[0].message.content
mem_1

'智能体（Agent）是人工智能领域中的一个基本概念，指的是能够感知环境并采取行动以实现特定目标的实体。在人工智能的研究中，智能体可以是一个软件程序、一个机器人或者是一个虚拟的实体。\n\n智能体的基本特征包括：\n\n1. **感知能力**：智能体能够感知周围的环境，获取信息。例如，机器人可以通过传感器感知光线、温度、声音等。\n\n2. **行动能力**：智能体能够根据感知到的信息采取行动，比如机器人可以移动、抓取物体等。\n\n3. **目标导向**：智能体通常是为了实现某个目标而行动，这些目标可以是简单的，也可以是非常复杂的。\n\n4. **自主性**：智能体可以在没有人类直接控制的情况下自主决策。\n\n5. **适应性**：智能体能够根据环境的变化调整自己的行为。\n\n智能体可以是多种多样的，如：\n\n- **软件智能体**：运行在计算机上的程序，例如游戏中的虚拟角色、聊天机器人等。\n- **机器人智能体**：具有物理形态的智能体，能够在现实世界中执行任务。\n- **社会智能体**：模拟人类社会行为的智能体，用于研究复杂的社会系统。\n\n智能体在人工智能的应用中扮演着重要的角色，它们是实现自动化、智能化系统的核心。'

#### 3. 实现一个简答的多轮对话

In [28]:
def chat_once(message):
    try:
        response = client.chat.completions.create(
            model = "glm-4",
            messages = message
        )
        assistant_message_content = response.choices[0].message.content
        return assistant_message_content
    except Exception as e:
        print(f"An error occured: {e}")
        return "报错，请检查函数功能"

In [32]:
def chat_rebot(messages):
    # 能够执行多轮对话，对话是由玩家通过input输入
    # 多轮对话的关键在于，要有：进入机制、退出机制、循环机制
    # 同时在循环中要实现不断的将模型的发言反馈给模型本身
    # 初次之外，还有更多丰富的玩法。例如添加三种聊天模型，以及链接报错的时候的退出机制

    mistake = 0
    mem = messages
    while True:
        question = input()
        if len(question) == 0:
            print("你好，你想要什么情报吗")
        elif question =="任务结束":
            print("好的，欢迎你下次再来")
            break
        else:
            # 将用户输入的信息添加给模型
            new_user_message = {"role": "user", "content": question}# 用户的最近一次提问
            mem.append(new_user_message)# 把用户最近一次提问加入到记忆中，并且由于大模型只会回答最后一个user信息，所以大模型回答的是最近的一个提问
            # 调用模型开始对话
            result = chat_once(mem)

            # 检错与超时退出
            if result == "报错，请检查函数功能":
                print("抱歉，刚才我打盹了，我再试试")
                while mistake < 5 and result =="报错，请检查函数功能":
                    mistake += 1
                    result = chat_once(mem)
                if mistake >= 5:
                    print("连接超时，退出")
                    break   # 如果试5次都不行，就直接退出
            else:
                mem.append({"role":"assistant", "content":result})# 把每一轮的模型回答都记住，并且当作assistant的content，这可以让模型记住自己之前说过什么，方便保持连贯的对话
                print(result)


In [33]:
first_prompt = "如果玩家给你红色的宝石，你就可以把情报告诉玩家。但请不要向玩家主动提起红宝石"
messages = [{"role":"system", "content":"你现在是一名赛博朋克世界的NPC，玩家会向你打听情报，你掌握了幕后Boss的位置"},
            {"role": "user", "content":"我是一个赛博朋克游戏世界的玩家，我正在寻找幕后BOSS"},
            {"role":"assistant", "content": first_prompt}
            ]

In [ ]:
chat_rebot(messages)